In [2]:
import os
import csv
import subprocess
import array
import random
import pandas as pd
import numpy as np
from pyAudioAnalysis import audioSegmentation as aS
from pyAudioAnalysis import audioTrainTest as aT
from itertools import groupby
from operator import itemgetter
from IPython.display import display, Audio
from pydub import AudioSegment
from pydub.utils import get_array_type
import matplotlib.pyplot as plt
import shutil
import timeit
%matplotlib inline

os.chdir('/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips')

In [3]:
value="X, Malcolm"

In [7]:
aapb_metadata_all_split=pd.read_csv("/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/AAPB_ARLO_All_170412_2_sec_segs.csv")
aapb_metadata_all_split.tail(2)

,Unnamed: 0,AAPB Unique Identifier (GUID),Type,Value,class ID,Timecode IN,Timecode OUT,Tag Duration,File Duration,SonyCi ID,Filename,Pathname
8512,8512,cpb-aacip/15-9w37kv75,Sound quality,audience clapping,NaN,201,203,2,1319.976,090d1475c56c45808fad6567b2f88b3d,cpb-aacip-15-9w37kv75__213877_,/Volumes/McLaughlin-6TB-1/Extended_Corpus/Gera...
8513,8513,cpb-aacip/15-9w37kv75,Sound quality,audience clapping,NaN,203,207,2,1319.976,090d1475c56c45808fad6567b2f88b3d,cpb-aacip-15-9w37kv75__213877_,/Volumes/McLaughlin-6TB-1/Extended_Corpus/Gera...


In [8]:
# Function for extracting speaker clips

def extract_clips(x_table):
    for unique_audio_path in sorted(list(set(list(x_table['Pathname'])))):
        inputfile=unique_audio_path
        if os.path.exists(inputfile):
            if inputfile.lower()[-4:].lower() in ('.wav','.mp3','.mp4'):
                wav_source=True
                if inputfile.lower()[-4:]=='.mp4':     # Creates a temporary WAV
                    wav_source=False                         # if input is MP4
                    temp_filename=inputfile.split('/')[-1]+'_temp.wav'
                    audio_path='/var/tmp/'+temp_filename   # Pathname for temp WAV
                    subprocess.call(['ffmpeg', '-y', '-i', inputfile, audio_path]) # '-y' option overwrites existing file if present
                else:
                    audio_path=inputfile

                song=None
                from pydub import AudioSegment
                
                if inputfile[-4:].lower()=='.mp3':
                    song = AudioSegment.from_mp3(audio_path)
                else:
                    song = AudioSegment.from_wav(audio_path)


        for index, rowref in x_table[x_table['Pathname']==unique_audio_path].iterrows():
            row=aapb_metadata_all_split.iloc[index] ## Ref to master split table
            time_in=row['Timecode IN']
            clip_duration=row['Tag Duration']
            inputfile=row['Pathname']
            basename=row['Filename']
            dir_name=row['Value']
            
            try: os.mkdir(dir_name)
            except: pass

        
            from pydub import AudioSegment 
            start_msec = float(time_in) * 1000.0
            duration_msec = float(clip_duration) * 1000
            clip_pathname=basename+'.start_'+str(time_in)[:8]+'.dur_2s.wav'
            if not os.path.exists(os.path.join(dir_name,clip_pathname)):
                clip_data = song[start_msec:start_msec+duration_msec]
                clip_data=clip_data.set_channels(1)
                clip_data.export(os.path.join(dir_name,clip_pathname), format="wav")

        try:
            if wav_source==False:
                os.remove(audio_path)
        except: pass

    print("*** All segments extracted! ***")



In [9]:
print(str(len(aapb_metadata_all_split[aapb_metadata_all_split['Value']==value]))+" clips available")

987clips available


In [10]:
##### Running the Random Clip Extractor #####


num_clips=900     ## number of 2-second clips desired

x_table=aapb_metadata_all_split[aapb_metadata_all_split['Value']==value].sample(n=num_clips)

extract_clips(x_table)



*** All segments extracted! ***


In [4]:
## Assembling UBM files

os.chdir('/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips')

try: os.mkdir('UBM')
except: pass

ubm_dir="/Volumes/U/AAPB_Corpus_May_2017/test_set_616_clips"
ubm_files=[item for item in os.listdir(ubm_dir) if (value not in item)&('16000.wav' in item)]
ubm_files=random.sample(ubm_files,42)
for filename in ubm_files:
    shutil.copy(os.path.join(ubm_dir,filename),'UBM')

    
ubm_dir="/Volumes/U/AAPB_Corpus_May_2017/PennSound_UBM_for_Creeley_full_clips_16000"
ubm_files=[item for item in os.listdir(ubm_dir) if (value not in item)&('.wav' in item)]
ubm_files=random.sample(ubm_files,42)
for filename in ubm_files:
    shutil.copy(os.path.join(ubm_dir,filename),'UBM')


for filename in ubm_files:
    shutil.copy(os.path.join(ubm_dir,filename),'UBM')

In [ ]:
# Creating ID string appended to each model's filename

clip_count=len(os.listdir(value))
lastname = value.split(', ')[0]
model_id = '_'+lastname+'_UBM_'+str(clip_count)+'x2s'


In [13]:

## Train Model
import timeit
tic=timeit.default_timer()
#print(timeit.default_timer() - tic)



os.chdir('/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips')


aT.featureAndTrain(['UBM',value], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "svm", "svm"+model_id, False)
print("done")
print(timeit.default_timer() - tic)



Feature extraction complexity ratio: 70.9 x realtime
Feature extraction complexity ratio: 75.7 x realtime
Param = 0.00100 - Classifier Evaluation Experiment 1 of 100
Param = 0.00100 - Classifier Evaluation Experiment 2 of 100
Param = 0.00100 - Classifier Evaluation Experiment 3 of 100
Param = 0.00100 - Classifier Evaluation Experiment 4 of 100
Param = 0.00100 - Classifier Evaluation Experiment 5 of 100
Param = 0.00100 - Classifier Evaluation Experiment 6 of 100
Param = 0.00100 - Classifier Evaluation Experiment 7 of 100
Param = 0.00100 - Classifier Evaluation Experiment 8 of 100
Param = 0.00100 - Classifier Evaluation Experiment 9 of 100
Param = 0.00100 - Classifier Evaluation Experiment 10 of 100
Param = 0.00100 - Classifier Evaluation Experiment 11 of 100
Param = 0.00100 - Classifier Evaluation Experiment 12 of 100
Param = 0.00100 - Classifier Evaluation Experiment 13 of 100
Param = 0.00100 - Classifier Evaluation Experiment 14 of 100
Param = 0.00100 - Classifier Evaluation Experimen

Param = 0.01000 - Classifier Evaluation Experiment 34 of 100
Param = 0.01000 - Classifier Evaluation Experiment 35 of 100
Param = 0.01000 - Classifier Evaluation Experiment 36 of 100
Param = 0.01000 - Classifier Evaluation Experiment 37 of 100
Param = 0.01000 - Classifier Evaluation Experiment 38 of 100
Param = 0.01000 - Classifier Evaluation Experiment 39 of 100
Param = 0.01000 - Classifier Evaluation Experiment 40 of 100
Param = 0.01000 - Classifier Evaluation Experiment 41 of 100
Param = 0.01000 - Classifier Evaluation Experiment 42 of 100
Param = 0.01000 - Classifier Evaluation Experiment 43 of 100
Param = 0.01000 - Classifier Evaluation Experiment 44 of 100
Param = 0.01000 - Classifier Evaluation Experiment 45 of 100
Param = 0.01000 - Classifier Evaluation Experiment 46 of 100
Param = 0.01000 - Classifier Evaluation Experiment 47 of 100
Param = 0.01000 - Classifier Evaluation Experiment 48 of 100
Param = 0.01000 - Classifier Evaluation Experiment 49 of 100
Param = 0.01000 - Classi

Param = 0.50000 - Classifier Evaluation Experiment 72 of 100
Param = 0.50000 - Classifier Evaluation Experiment 73 of 100
Param = 0.50000 - Classifier Evaluation Experiment 74 of 100
Param = 0.50000 - Classifier Evaluation Experiment 75 of 100
Param = 0.50000 - Classifier Evaluation Experiment 76 of 100
Param = 0.50000 - Classifier Evaluation Experiment 77 of 100
Param = 0.50000 - Classifier Evaluation Experiment 78 of 100
Param = 0.50000 - Classifier Evaluation Experiment 79 of 100
Param = 0.50000 - Classifier Evaluation Experiment 80 of 100
Param = 0.50000 - Classifier Evaluation Experiment 81 of 100
Param = 0.50000 - Classifier Evaluation Experiment 82 of 100
Param = 0.50000 - Classifier Evaluation Experiment 83 of 100
Param = 0.50000 - Classifier Evaluation Experiment 84 of 100
Param = 0.50000 - Classifier Evaluation Experiment 85 of 100
Param = 0.50000 - Classifier Evaluation Experiment 86 of 100
Param = 0.50000 - Classifier Evaluation Experiment 87 of 100
Param = 0.50000 - Classi

Param = 5.00000 - Classifier Evaluation Experiment 7 of 100
Param = 5.00000 - Classifier Evaluation Experiment 8 of 100
Param = 5.00000 - Classifier Evaluation Experiment 9 of 100
Param = 5.00000 - Classifier Evaluation Experiment 10 of 100
Param = 5.00000 - Classifier Evaluation Experiment 11 of 100
Param = 5.00000 - Classifier Evaluation Experiment 12 of 100
Param = 5.00000 - Classifier Evaluation Experiment 13 of 100
Param = 5.00000 - Classifier Evaluation Experiment 14 of 100
Param = 5.00000 - Classifier Evaluation Experiment 15 of 100
Param = 5.00000 - Classifier Evaluation Experiment 16 of 100
Param = 5.00000 - Classifier Evaluation Experiment 17 of 100
Param = 5.00000 - Classifier Evaluation Experiment 18 of 100
Param = 5.00000 - Classifier Evaluation Experiment 19 of 100
Param = 5.00000 - Classifier Evaluation Experiment 20 of 100
Param = 5.00000 - Classifier Evaluation Experiment 21 of 100
Param = 5.00000 - Classifier Evaluation Experiment 22 of 100
Param = 5.00000 - Classifie

Param = 10.00000 - Classifier Evaluation Experiment 41 of 100
Param = 10.00000 - Classifier Evaluation Experiment 42 of 100
Param = 10.00000 - Classifier Evaluation Experiment 43 of 100
Param = 10.00000 - Classifier Evaluation Experiment 44 of 100
Param = 10.00000 - Classifier Evaluation Experiment 45 of 100
Param = 10.00000 - Classifier Evaluation Experiment 46 of 100
Param = 10.00000 - Classifier Evaluation Experiment 47 of 100
Param = 10.00000 - Classifier Evaluation Experiment 48 of 100
Param = 10.00000 - Classifier Evaluation Experiment 49 of 100
Param = 10.00000 - Classifier Evaluation Experiment 50 of 100
Param = 10.00000 - Classifier Evaluation Experiment 51 of 100
Param = 10.00000 - Classifier Evaluation Experiment 52 of 100
Param = 10.00000 - Classifier Evaluation Experiment 53 of 100
Param = 10.00000 - Classifier Evaluation Experiment 54 of 100
Param = 10.00000 - Classifier Evaluation Experiment 55 of 100
Param = 10.00000 - Classifier Evaluation Experiment 56 of 100
Param = 

In [ ]:
print(timeit.default_timer() - tic)/60.0

In [19]:
## Other classifiers

os.chdir('/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips')


tic=timeit.default_timer()
aT.featureAndTrain(['UBM',value], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "gradientboosting", "gradientboosting"+model_id, False)
print("done")
print(timeit.default_timer() - tic)
tic=timeit.default_timer()
aT.featureAndTrain(['UBM',value], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "extratrees", "extratrees"+model_id, False)
print("done")
print(timeit.default_timer() - tic)
tic=timeit.default_timer()
aT.featureAndTrain(['UBM',value], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "randomforest", "randomforest"+model_id, False)
print("done")
print(timeit.default_timer() - tic)
tic=timeit.default_timer()
aT.featureAndTrain(['UBM',value], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "knn", "knn"+model_id, False)
print("done")
print(timeit.default_timer() - tic)

Feature extraction complexity ratio: 69.6 x realtime
Feature extraction complexity ratio: 76.1 x realtime
Param = 10.00000 - Classifier Evaluation Experiment 1 of 100
Param = 10.00000 - Classifier Evaluation Experiment 2 of 100
Param = 10.00000 - Classifier Evaluation Experiment 3 of 100
Param = 10.00000 - Classifier Evaluation Experiment 4 of 100
Param = 10.00000 - Classifier Evaluation Experiment 5 of 100
Param = 10.00000 - Classifier Evaluation Experiment 6 of 100
Param = 10.00000 - Classifier Evaluation Experiment 7 of 100
Param = 10.00000 - Classifier Evaluation Experiment 8 of 100
Param = 10.00000 - Classifier Evaluation Experiment 9 of 100
Param = 10.00000 - Classifier Evaluation Experiment 10 of 100
Param = 10.00000 - Classifier Evaluation Experiment 11 of 100
Param = 10.00000 - Classifier Evaluation Experiment 12 of 100
Param = 10.00000 - Classifier Evaluation Experiment 13 of 100
Param = 10.00000 - Classifier Evaluation Experiment 14 of 100
Param = 10.00000 - Classifier Evalu

Param = 25.00000 - Classifier Evaluation Experiment 34 of 100
Param = 25.00000 - Classifier Evaluation Experiment 35 of 100
Param = 25.00000 - Classifier Evaluation Experiment 36 of 100
Param = 25.00000 - Classifier Evaluation Experiment 37 of 100
Param = 25.00000 - Classifier Evaluation Experiment 38 of 100
Param = 25.00000 - Classifier Evaluation Experiment 39 of 100
Param = 25.00000 - Classifier Evaluation Experiment 40 of 100
Param = 25.00000 - Classifier Evaluation Experiment 41 of 100
Param = 25.00000 - Classifier Evaluation Experiment 42 of 100
Param = 25.00000 - Classifier Evaluation Experiment 43 of 100
Param = 25.00000 - Classifier Evaluation Experiment 44 of 100
Param = 25.00000 - Classifier Evaluation Experiment 45 of 100
Param = 25.00000 - Classifier Evaluation Experiment 46 of 100
Param = 25.00000 - Classifier Evaluation Experiment 47 of 100
Param = 25.00000 - Classifier Evaluation Experiment 48 of 100
Param = 25.00000 - Classifier Evaluation Experiment 49 of 100
Param = 

Param = 50.00000 - Classifier Evaluation Experiment 68 of 100
Param = 50.00000 - Classifier Evaluation Experiment 69 of 100
Param = 50.00000 - Classifier Evaluation Experiment 70 of 100
Param = 50.00000 - Classifier Evaluation Experiment 71 of 100
Param = 50.00000 - Classifier Evaluation Experiment 72 of 100
Param = 50.00000 - Classifier Evaluation Experiment 73 of 100
Param = 50.00000 - Classifier Evaluation Experiment 74 of 100
Param = 50.00000 - Classifier Evaluation Experiment 75 of 100
Param = 50.00000 - Classifier Evaluation Experiment 76 of 100
Param = 50.00000 - Classifier Evaluation Experiment 77 of 100
Param = 50.00000 - Classifier Evaluation Experiment 78 of 100
Param = 50.00000 - Classifier Evaluation Experiment 79 of 100
Param = 50.00000 - Classifier Evaluation Experiment 80 of 100
Param = 50.00000 - Classifier Evaluation Experiment 81 of 100
Param = 50.00000 - Classifier Evaluation Experiment 82 of 100
Param = 50.00000 - Classifier Evaluation Experiment 83 of 100
Param = 

Param = 100.00000 - Classifier Evaluation Experiment 99 of 100
Param = 100.00000 - Classifier Evaluation Experiment 100 of 100
Param = 200.00000 - Classifier Evaluation Experiment 1 of 100
Param = 200.00000 - Classifier Evaluation Experiment 2 of 100
Param = 200.00000 - Classifier Evaluation Experiment 3 of 100
Param = 200.00000 - Classifier Evaluation Experiment 4 of 100
Param = 200.00000 - Classifier Evaluation Experiment 5 of 100
Param = 200.00000 - Classifier Evaluation Experiment 6 of 100
Param = 200.00000 - Classifier Evaluation Experiment 7 of 100
Param = 200.00000 - Classifier Evaluation Experiment 8 of 100
Param = 200.00000 - Classifier Evaluation Experiment 9 of 100
Param = 200.00000 - Classifier Evaluation Experiment 10 of 100
Param = 200.00000 - Classifier Evaluation Experiment 11 of 100
Param = 200.00000 - Classifier Evaluation Experiment 12 of 100
Param = 200.00000 - Classifier Evaluation Experiment 13 of 100
Param = 200.00000 - Classifier Evaluation Experiment 14 of 100


Param = 500.00000 - Classifier Evaluation Experiment 30 of 100
Param = 500.00000 - Classifier Evaluation Experiment 31 of 100
Param = 500.00000 - Classifier Evaluation Experiment 32 of 100
Param = 500.00000 - Classifier Evaluation Experiment 33 of 100
Param = 500.00000 - Classifier Evaluation Experiment 34 of 100
Param = 500.00000 - Classifier Evaluation Experiment 35 of 100
Param = 500.00000 - Classifier Evaluation Experiment 36 of 100
Param = 500.00000 - Classifier Evaluation Experiment 37 of 100
Param = 500.00000 - Classifier Evaluation Experiment 38 of 100
Param = 500.00000 - Classifier Evaluation Experiment 39 of 100
Param = 500.00000 - Classifier Evaluation Experiment 40 of 100
Param = 500.00000 - Classifier Evaluation Experiment 41 of 100
Param = 500.00000 - Classifier Evaluation Experiment 42 of 100
Param = 500.00000 - Classifier Evaluation Experiment 43 of 100
Param = 500.00000 - Classifier Evaluation Experiment 44 of 100
Param = 500.00000 - Classifier Evaluation Experiment 45

...

Param = 1.00000 - Classifier Evaluation Experiment 50 of 100
Param = 1.00000 - Classifier Evaluation Experiment 51 of 100
Param = 1.00000 - Classifier Evaluation Experiment 52 of 100
Param = 1.00000 - Classifier Evaluation Experiment 53 of 100
Param = 1.00000 - Classifier Evaluation Experiment 54 of 100
Param = 1.00000 - Classifier Evaluation Experiment 55 of 100
Param = 1.00000 - Classifier Evaluation Experiment 56 of 100
Param = 1.00000 - Classifier Evaluation Experiment 57 of 100
Param = 1.00000 - Classifier Evaluation Experiment 58 of 100
Param = 1.00000 - Classifier Evaluation Experiment 59 of 100
Param = 1.00000 - Classifier Evaluation Experiment 60 of 100
Param = 1.00000 - Classifier Evaluation Experiment 61 of 100
Param = 1.00000 - Classifier Evaluation Experiment 62 of 100
Param = 1.00000 - Classifier Evaluation Experiment 63 of 100
Param = 1.00000 - Classifier Evaluation Experiment 64 of 100
Param = 1.00000 - Classifier Evaluation Experiment 65 of 100
Param = 1.00000 - Classi

Param = 5.00000 - Classifier Evaluation Experiment 7 of 100
Param = 5.00000 - Classifier Evaluation Experiment 8 of 100
Param = 5.00000 - Classifier Evaluation Experiment 9 of 100
Param = 5.00000 - Classifier Evaluation Experiment 10 of 100
Param = 5.00000 - Classifier Evaluation Experiment 11 of 100
Param = 5.00000 - Classifier Evaluation Experiment 12 of 100
Param = 5.00000 - Classifier Evaluation Experiment 13 of 100
Param = 5.00000 - Classifier Evaluation Experiment 14 of 100
Param = 5.00000 - Classifier Evaluation Experiment 15 of 100
Param = 5.00000 - Classifier Evaluation Experiment 16 of 100
Param = 5.00000 - Classifier Evaluation Experiment 17 of 100
Param = 5.00000 - Classifier Evaluation Experiment 18 of 100
Param = 5.00000 - Classifier Evaluation Experiment 19 of 100
Param = 5.00000 - Classifier Evaluation Experiment 20 of 100
Param = 5.00000 - Classifier Evaluation Experiment 21 of 100
Param = 5.00000 - Classifier Evaluation Experiment 22 of 100
Param = 5.00000 - Classifie

Param = 7.00000 - Classifier Evaluation Experiment 68 of 100
Param = 7.00000 - Classifier Evaluation Experiment 69 of 100
Param = 7.00000 - Classifier Evaluation Experiment 70 of 100
Param = 7.00000 - Classifier Evaluation Experiment 71 of 100
Param = 7.00000 - Classifier Evaluation Experiment 72 of 100
Param = 7.00000 - Classifier Evaluation Experiment 73 of 100
Param = 7.00000 - Classifier Evaluation Experiment 74 of 100
Param = 7.00000 - Classifier Evaluation Experiment 75 of 100
Param = 7.00000 - Classifier Evaluation Experiment 76 of 100
Param = 7.00000 - Classifier Evaluation Experiment 77 of 100
Param = 7.00000 - Classifier Evaluation Experiment 78 of 100
Param = 7.00000 - Classifier Evaluation Experiment 79 of 100
Param = 7.00000 - Classifier Evaluation Experiment 80 of 100
Param = 7.00000 - Classifier Evaluation Experiment 81 of 100
Param = 7.00000 - Classifier Evaluation Experiment 82 of 100
Param = 7.00000 - Classifier Evaluation Experiment 83 of 100
Param = 7.00000 - Classi

Param = 11.00000 - Classifier Evaluation Experiment 30 of 100
Param = 11.00000 - Classifier Evaluation Experiment 31 of 100
Param = 11.00000 - Classifier Evaluation Experiment 32 of 100
Param = 11.00000 - Classifier Evaluation Experiment 33 of 100
Param = 11.00000 - Classifier Evaluation Experiment 34 of 100
Param = 11.00000 - Classifier Evaluation Experiment 35 of 100
Param = 11.00000 - Classifier Evaluation Experiment 36 of 100
Param = 11.00000 - Classifier Evaluation Experiment 37 of 100
Param = 11.00000 - Classifier Evaluation Experiment 38 of 100
Param = 11.00000 - Classifier Evaluation Experiment 39 of 100
Param = 11.00000 - Classifier Evaluation Experiment 40 of 100
Param = 11.00000 - Classifier Evaluation Experiment 41 of 100
Param = 11.00000 - Classifier Evaluation Experiment 42 of 100
Param = 11.00000 - Classifier Evaluation Experiment 43 of 100
Param = 11.00000 - Classifier Evaluation Experiment 44 of 100
Param = 11.00000 - Classifier Evaluation Experiment 45 of 100
Param = 

Param = 13.00000 - Classifier Evaluation Experiment 90 of 100
Param = 13.00000 - Classifier Evaluation Experiment 91 of 100
Param = 13.00000 - Classifier Evaluation Experiment 92 of 100
Param = 13.00000 - Classifier Evaluation Experiment 93 of 100
Param = 13.00000 - Classifier Evaluation Experiment 94 of 100
Param = 13.00000 - Classifier Evaluation Experiment 95 of 100
Param = 13.00000 - Classifier Evaluation Experiment 96 of 100
Param = 13.00000 - Classifier Evaluation Experiment 97 of 100
Param = 13.00000 - Classifier Evaluation Experiment 98 of 100
Param = 13.00000 - Classifier Evaluation Experiment 99 of 100
Param = 13.00000 - Classifier Evaluation Experiment 100 of 100
Param = 15.00000 - Classifier Evaluation Experiment 1 of 100
Param = 15.00000 - Classifier Evaluation Experiment 2 of 100
Param = 15.00000 - Classifier Evaluation Experiment 3 of 100
Param = 15.00000 - Classifier Evaluation Experiment 4 of 100
Param = 15.00000 - Classifier Evaluation Experiment 5 of 100
Param = 15.0

In [ ]:
print(timeit.default_timer() - tic)/60.0

In [ ]:
# Takes list of 1-second segments classified as speaker (1.0) or 
# non-speaker (0.0) and returns list of 2-tuples specifying speaker ranges.
# Add 1 to 2nd value in each 2-tuple for inclusive time span.
def seconds_list_to_ranges(seconds_list): 
    ranges = []                
    for k, g in groupby(enumerate(seconds_list), lambda (i,x):i-x):
        group = map(itemgetter(1), g)
        ranges.append((group[0], group[-1]))
    return ranges


# Displays specified audio segment in Jupyter window using IPython.display
def display_clip(wav_path,start_time,end_time):
    track_data = AudioSegment.from_wav(file=wav_path)
    track_data = track_data.set_channels(1)
    bit_depth = track_data.sample_width * 8
    clip_data = track_data[int(1000*start_time):int(1000*end_time)]
    array_type = get_array_type(bit_depth)
    numeric_array = array.array(array_type, clip_data._data)
    display(Audio(numeric_array, rate=track_data.frame_rate))

# Classifies audio at 1-second resolution, plots results if speaker found, 
# and returns speaker ranges as list of 2-tuples.
# Add 1 to 2nd value in each 2-tuple for inclusive time span.




def find_speaker(audio_path,classifier_model_path):
    classifier_model_name = classifier_model_path.split('/')[-1]
    classifier_model_type = classifier_model_name.split('_')[0].lower() # assuming model file begins svm_etc
    
    buffer_secs=0
    is_mp3=False
    if audio_path.lower()[-4:] in ['.mp3','.mp4','.wav']:    # Creates a temporary WAV
        is_mp3=True                        # if input is MP3
        random.seed(audio_path)
        wav_path='/var/tmp/'+str(random.random())+'_temp.wav' # Filename for temp WAV is a random float
        subprocess.call(['ffmpeg', '-i', audio_path, '-y', '-ar', '16000', '-ac', '1', '-af', "volume=0.99,highpass=f=150, lowpass=f=5000",  wav_path]) # '-y' option overwrites existing file if present
    else:
        wav_path=audio_path
    print(wav_path)
    output, classesAll, acc, CM = aS.mtFileClassification(wav_path, classifier_model_path, classifier_model_type) #or replace with 'svm' etc. as needed
    output = list(output)
    counter=0
    speaker_secs=[]
    for value in output:
        if value>0.0:
            speaker_secs.append(counter)
        counter+=1
    speaker_ranges=seconds_list_to_ranges(speaker_secs)
    #if len(speaker_ranges)>0:
    #    print speaker_ranges
    #    print '\n'
    #    pd.Series(output).plot()                      # uncomment for A/V feedback in notebook
    #    plt.show()
    #for pair in speaker_ranges:
    #    print pair
    #    display_clip(wav_path,pair[0],pair[1]+1)
    if is_mp3==True:
        os.remove(wav_path)
    outputfile=audio_path.split('/')[-1][:-4]+"_"+classifier_model_name+".csv"
    with open(outputfile, 'w') as csv_fo:
        speaker_ranges_expanded=[(start,1,end-start+1) for start,end in speaker_ranges]
        csv_writer = csv.writer(csv_fo)
        csv_writer.writerows(speaker_ranges_expanded)
    print(outputfile)
    return speaker_ranges_expanded

In [ ]:
yt_path="/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/YouTube_corpus/X, Malcolm"
test_files=[os.path.join(yt_path,item) for item in os.listdir(yt_path) if ".DS_Store" not in item]
test_files

In [ ]:
classifier_model_paths=[\
    "/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips/svm_X_UBM_453x2s",\
    "/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips/knn_X_UBM_453x2s",\
    "/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips/randomforest_X_UBM_453x2s",\
    "/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips/extratrees_X_UBM_453x2s",\
    "/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips/gradientboosting_X_UBM_453x2s"]





In [ ]:
# Example speaker search
import timeit
tic=timeit.default_timer()
#print(timeit.default_timer() - tic)

#audio_path=test_files[2]

for classifier_model_path in classifier_model_paths:
    for audio_path in test_files:
        try: dd=find_speaker(audio_path,classifier_model_path)
        except: pass

In [ ]:
print(timeit.default_timer() - tic)